# Exif Digger

EXIF, which stands for Exchangeable Image File Format, is a standard that embeds metadata within image files. This metadata includes various details about the image, such as the date and time it was taken, camera settings, location information, and even the make and model of the camera used. Essentially, EXIF data acts as a digital fingerprint for photographs, providing a wealth of information that can be important in forensic investigations.

In [62]:
import folium
from exif import Image
from IPython.display import HTML, display

colors = {'PURPLE':'\033[95m', 'CYAN':'\033[96m', 'DARKCYAN':'\033[36m', 'BLUE':'\033[94m',
          'GREEN':'\033[92m', 'YELLOW':'\033[93m', 'RED':'\033[91m','BOLD':'\033[1m',
          'UNDERLINE':'\033[4m', 'END':'\033[0m'}

In [63]:
img_path = "imgs/sample1.jpg"
with open(img_path, "rb") as f:
    img = Image(f)

In [64]:
def get_model(img):
    try:
        if "model" in dir(img):
            return img.model
    except Exception as e:
        print(e)

In [65]:
def get_datetimes(img):
    try:
        if "datetime_original" in dir(img) and "datetime" in dir(img):
            return img.datetime_original, img.datetime
    except Exception as e:
        print(e)

In [66]:
def get_lat_long(img):
    try:
        if "gps_latitude" in dir(img) and "gps_longitude" in dir(img):
            latitude = img.gps_latitude
            longitude = img.gps_longitude
            direction_lat = img.gps_latitude_ref
            direction_long = img.gps_longitude_ref

            float_lat = (
                float(latitude[0]) + float(latitude[1]) / 60 + float(latitude[2]) / 3600
            )
            float_long = (
                float(longitude[0])
                + float(longitude[1]) / 60
                + float(longitude[2]) / 3600
            )

            if direction_lat == "S":
                float_lat = -float_lat
            if direction_long == "W":
                float_long = -float_long

            return float_lat, float_long
    except Exception as e:
        print(e)

In [74]:
def report(model, created, modified, lat, long, img_path):
    try:
        print(colors["BOLD"] + "################# Image Exif Report ################" + colors["END"])
        print(colors["PURPLE"] + "Image taken with device: " + colors["END"] + model)
        if (created == modified) or modified is None:
            print(colors["GREEN"] + "No modification detected." + colors["END"])
            print(colors["YELLOW"] + "Created: " + colors["END"] + created)
        else:
            print(colors["RED"] + "Modification detected." + colors["END"])
            print(colors["YELLOW"] + "Created: " + colors["END"] + created)
            print(colors["RED"] + "Modified: " + colors["END"] + modified)

        if lat is not None and long is not None:
            f = folium.Figure(width=300, height=300)
            m = folium.Map(location=[lat, long], zoom_start=15)
            folium.Marker([lat, long], icon=folium.Icon(color="red")).add_to(m)
            f.add_child(m)
            return f
        else:
            print(colors["RED"] + "No GPS data found." + colors["END"])
    except Exception as e:
        print(e)

In [75]:
if img.has_exif:
    model = get_model(img)
    created, modified = get_datetimes(img)
    lat, long = get_lat_long(img)
    m = report(model, created, modified, lat, long, img_path)
    display(m)
    display(HTML(f'<img src="{img_path}" width="300">'))

################# Image Exif Report ################
Image taken with device: COOLPIX P6000
Modification detected.
Created: 2008:10:22 16:28:39
Modified: 2008:11:01 21:15:07
